# FedExCharge_2024

In [1]:
import pandas as pd
import numpy as np

FedExClient_2024 = pd.read_excel('C:\\Users\\sliu\\BD\\re-rate_model\\1-2-1.FedExClient_2024.xlsx')
FedExDiscount_2024 = pd.read_excel('C:\\Users\\sliu\\BD\\re-rate_model\\1-2-2.FedExDiscount_2024.xlsx')
UPSCharge_2024 = pd.read_excel('C:\\Users\\sliu\\BD\\re-rate_model\\1-1.UPSCharge_2024.xlsx')


In [2]:
# Adding a temporary key column for joining
FedExClient_2024['key'] = 1
FedExDiscount_2024['key'] = 1

# Performing the cross join
FedExCharge_2024 = pd.merge(FedExClient_2024, FedExDiscount_2024, on='key').drop('key', axis=1)


In [3]:
FedExCharge_2024 = FedExCharge_2024.merge(
    UPSCharge_2024[["UPS ACCT #", "CompanyID", "Company", "UPSServiceCode", "UPSServiceName", "UPSServiceNote", "UPSMin", "UPSPrice", "PriceType", "StartDate", "EndDate"]],
    left_on=['CompanyID', 'UPSServiceCode', 'UPSServiceNote'],
    right_on=['CompanyID', 'UPSServiceCode', 'UPSServiceNote'],
    how='left',
    suffixes=('_x', '_y')  # Explicitly state suffixes to handle duplicate column names
)

# Rename 'Company_x' to 'Company' and drop 'Company_y'
FedExCharge_2024 = FedExCharge_2024.rename(columns={'Company_x': 'Company'}).drop('Company_y', axis=1)
FedExCharge_2024 = FedExCharge_2024.rename(columns={'UPSServiceName_x': 'UPSServiceName'}).drop('UPSServiceName_y', axis=1)


In [4]:
FedExCharge_2024.columns

Index(['CompanyID', 'Company', 'FedExServiceCode', 'FedExServiceName',
       'FedExServiceNote', 'UPSServiceCode', 'UPSServiceName',
       'UPSServiceNote', 'FURatio', 'UPS ACCT #', 'UPSMin', 'UPSPrice',
       'PriceType', 'StartDate', 'EndDate'],
      dtype='object')

In [5]:
# Create a mask for the specific conditions
mask1 = (FedExCharge_2024['FedExServiceCode'] == 'SPST') & (FedExCharge_2024['FedExServiceNote'] == '4-70lbs')

# Calculate the mean price only for the filtered rows
FedExCharge_2024.loc[mask1, 'FedExPrice'] = FedExCharge_2024.loc[mask1].groupby(['CompanyID', 'Company', 'FedExServiceCode', 'FedExServiceName', 'FedExServiceNote'])['UPSPrice'].transform('mean')

FedExCharge_2024.loc[mask1, 'FedExPrice'] = round(1-(1-FedExCharge_2024['FedExPrice'])/FedExCharge_2024['FURatio'],3)


In [6]:

# Create a mask for the specific conditions
mask2 = ~((FedExCharge_2024['FedExServiceCode'] == 'SPST') & (FedExCharge_2024['FedExServiceNote'] == '4-70lbs'))

FedExCharge_2024.loc[mask2, 'FedExPrice'] = round(1-(1-FedExCharge_2024['UPSPrice'])/FedExCharge_2024['FURatio'],3)


In [7]:

# Create a mask for the specific conditions
mask3 = FedExCharge_2024['FURatio'].isna() & FedExCharge_2024['FedExServiceCode'].notna()

FedExCharge_2024.loc[mask3, 'FedExPrice'] = FedExCharge_2024['UPSPrice']

In [8]:

# Define the function to update the 'New' value within each group
def update_new(group):
    # Get the 'FedExPrice' value where 'FedExServiceCode' is 'DASFCG' within the group
    target_new_value = group.loc[group['FedExServiceCode'] == 'DASFCG', 'FedExPrice']
    if not target_new_value.empty:
        target_new_value = target_new_value.iloc[0]  # Take the first if there are multiple
        # Set 'FedExPrice' where 'FedExServiceCode' is in the specified list
        mask4 = group['FedExServiceCode'].isin(['FREM', 'FREMAK', 'FREMHI'])
        group.loc[mask4, 'FedExPrice'] = target_new_value
    return group

# Apply the function to each group formed by 'CompanyID'
FedExCharge_2024 = FedExCharge_2024.groupby('CompanyID').apply(update_new)


C:\Users\sliu\AppData\Local\Temp\ipykernel_13884\3283895371.py:13: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  FedExCharge_2024 = FedExCharge_2024.groupby('CompanyID').apply(update_new)


In [9]:
FedExCharge_2024.columns

Index(['CompanyID', 'Company', 'FedExServiceCode', 'FedExServiceName',
       'FedExServiceNote', 'UPSServiceCode', 'UPSServiceName',
       'UPSServiceNote', 'FURatio', 'UPS ACCT #', 'UPSMin', 'UPSPrice',
       'PriceType', 'StartDate', 'EndDate', 'FedExPrice'],
      dtype='object')

In [10]:

# Specify the columns to be dropped
columns_to_drop = ['UPSServiceCode', 'UPSServiceName',
       'UPSServiceNote', 'FURatio', 'UPSPrice']

# Drop the columns from the DataFrame
FedExCharge_2024 = FedExCharge_2024.drop(columns=columns_to_drop)


In [11]:
# Specify the new order of your columns
new_order = ['UPS ACCT #', 'CompanyID', 'Company', 'FedExServiceCode', 'FedExServiceName',
       'FedExServiceNote', 'UPSMin', 'FedExPrice', 'PriceType', 'StartDate', 'EndDate'] 

# Reorder the columns using the new order
FedExCharge_2024 = FedExCharge_2024[new_order]


In [12]:
FedExCharge_2024.columns

Index(['UPS ACCT #', 'CompanyID', 'Company', 'FedExServiceCode',
       'FedExServiceName', 'FedExServiceNote', 'UPSMin', 'FedExPrice',
       'PriceType', 'StartDate', 'EndDate'],
      dtype='object')

In [13]:
# Rename columns 
FedExCharge_2024 = FedExCharge_2024.rename(columns={
    'UPSMin': 'FedExMin'
})

In [14]:
# Drop columns that are completely blank
FedExCharge_2024 = FedExCharge_2024.dropna(axis=1, how='all')

# Drop rows that are completely blank
FedExCharge_2024 = FedExCharge_2024.dropna(axis=0, how='all')


In [15]:
# Drop columns that are duplicated. 
FedExCharge_2024 = FedExCharge_2024.drop_duplicates()

In [16]:

# Specify the values to filter out
values_to_remove = [
    'FedEx Priority Express Import', 
    'FedEx Priority Import', 
    'FedEx Economy Import'
]

# Create a mask that is True for rows where 'Service' is not in the values_to_remove
mask = ~FedExCharge_2024['FedExServiceName'].isin(values_to_remove)

# Use the mask to filter the DataFrame
FedExCharge_2024 = FedExCharge_2024[mask]



In [17]:

# Drop rows where 'FedexServiceCode' is null
FedExCharge_2024 = FedExCharge_2024.dropna(subset=['FedExServiceCode'])


In [18]:
FedExCharge_2024.to_excel('FedExCharge_2024.xlsx')

# ZipToZone

In [63]:
xls = pd.ExcelFile('6-1.ZipToZoneUPS.xlsx')

# List of sheet names. Using a dictionary comprehension to load sheets into a dictionary of dataframes
sheets = [
    '02038_Outbound', '08873_Outbound', '21226_Outbound', '38141_Outbound', '38654_Outbound', 
    '75041_Outbound', '90640_Outbound', '02324_Outbound', '06096_Outbound', '18105_Outbound', 
    '21240_Outbound', '43004_Outbound'
]

dfs = {sheet: pd.read_excel(xls, sheet) for sheet in sheets}

ZipToZoneUPS = pd.concat([dfs['02038_Outbound'], dfs['08873_Outbound'], dfs['21226_Outbound'], dfs['38141_Outbound'], dfs['38654_Outbound'], 
                       dfs['75041_Outbound'], dfs['90640_Outbound'], dfs['02324_Outbound'], dfs['06096_Outbound'], dfs['18105_Outbound'], 
                       dfs['21240_Outbound'], dfs['43004_Outbound']])

ZipToZoneUPS.to_csv('ZipToZoneUPS.csv')

In [64]:
xlsf = pd.ExcelFile('6-2.ZipToZoneFedex.xlsx')

# List of sheet names
sheetsf = [
    'TDR29112023_18101_FXG', 'TDR29112023_18101_FHD', 'TDR14122023_90640_FXG', 'TDR14122023_90640_FHD', 'TDR14122023_75041_FXG', 
    'TDR14122023_75041_FHD', 'TDR14122023_38654_FXG', 'TDR14122023_38654_FHD', 'TDR14122023_38141_FXG', 'TDR14122023_38141_FHD', 
    'TDR14122023_21226_FXG', 'TDR14122023_21226_FHD', 'TDR14122023_08873_FXG', 'TDR14122023_08873_FHD', 'TDR14122023_02038_FXG',
    'TDR14122023_02038_FHD'
]

# Load each sheet into a DataFrame and add a 'FileName' column to identify each sheet
dfsf = {sheet: pd.read_excel(xlsf, sheet_name=sheet).assign(FileName=sheet) for sheet in sheetsf}

# Concatenate all DataFrames together
ZipToZoneFedex = pd.concat(dfsf.values(), ignore_index=True)

ZipToZoneFedex.to_csv('ZipToZoneFedex.csv')